<center><a href="https://www.nvidia.cn/training/"><img src="https://dli-lms.s3.amazonaws.com/assets/general/DLI_Header_White.png" width="400" height="186" /></a></center>

# <font color="#76b900"> **6:** GenAI 服务简介</font>

在 notebook 4 中，我们构建了解码器和编码-解码器模型，以实现生成系统。在 notebook 5 中，我们转向多模态上下文进一步强化了它们，这扩展到了新的表示和架构。在这个 notebook 中，我们将把注意力转回文本生成，展示更强大的能力。

#### **学习目标：**
- 认识到基本生成模型在生产环境中的局限性。
- 理解 **LLM 和 GenAI 服务**的概念和优势。
- 学习如何部署和与 **vLLM HuggingFace 模型服务**交互。
- 探索与服务器部署相关的**高效和并发推理**方法。
- 学习使用 **NVIDIA NIM** 优化 LLM 部署的技术，用于实际应用。
- 了解**使用 LangChain 的 LLM 编排（orchestration）**，在接下来的 notebook 中将进一步展开。

<hr>
<br>

## **6.1：** 将模型扩展到现实世界用例

虽然我们在探索基础 GenAI 架构的直觉和结构方面进展的相当不错，但仍有几个关键限制让我们无法实现生产就绪的应用。

> #### **生成模型性能不足。**
> 
> 由于我们受到资源的限制，并想要调查各种组件，不得不将自己限制在较小的系统上。其中一些模型，包括一些领域特定的编码器和编码-解码器模型，实际上在某些上下文中能执行相当复杂的现实任务。然而目前为止，我们的模型在严肃的文本生成和跟随指令方面都有所不足。

> #### **推理效率低下。**
> 
> 我们一直专注于方法和直觉，了解发生了什么并对过程按预期运行感到满意。在现实世界的上下文中，这并不足够，因为速度很重要，多个过程必须结合在一起来满足要求。我们不能不管性能的问题。

> #### **部署是单用户单实例。**
>
> 我们一直依赖于对这些模型进行独占访问，来展示它们的内部工作流定义并观察实际发生的事情。然而，到目前为止，所有的部署都都隔离在您的 Jupyter notebook 环境中。这种访问和控制的级别无法扩展到大用户的场景中，因为对每个用户单独启动的成本很高，且共享访问 Python 对象很难管理。

在这个 notebook 中，我们将探索使用**推理服务**来部署和访问更通用的解码器风格模型，这些模型能够执行复杂任务，例如任意指令跟随和对话。

<hr>
<br>

## **6.2：访问您的第一个 LLM 服务**

在接下来的课程中，您将能够与三个不同的 LLM 服务部署进行交互，这些部署涵盖了不同的规模和考量。首先，让我们介绍一下最轻量级的选项。

### **vLLM HuggingFace 模型服务** 

[**vLLM 项目（虚拟大语言模型的简称）**](https://github.com/vllm-project/vllm) 是一个流行的开源 LLM 服务项目，支持多种 HuggingFace 模型。一旦配置为服务所选择的模型，vLLM 服务器将会：

- 下载或缓存模型及相关配置到可访问的目录中。

- **[可选]** 将模型缓存转换为更优化的形式，例如，使用低精度权重。

- 根据下载的配置将模型加载到可微分工作流中。

- **[可选]** 用更优化的接口替换工作流中的某些部分（例如，融合组件，功能相似但前向传递更快）。

- **创建并暴露连接路由，以标准、独立和可扩展的方式访问模型。**
    - **标准：** 接口应该得到良好的定义，并在相似模型的类别中共享。这允许用户在不同模型之间切换，并创建可以在客户端运行并可靠地操作的连接器。
    - **独立：** 接口在被其他用户使用时不应该表现得有所不同（除非必要，例如，负载时变慢）。虽然可以创建有状态的系统，实现一些缓存和优化，但大多数 GenAI 服务配置应保持无状态。
    - **可扩展：** 接口应该假设多个进程可能同时使用其入口，并努力保持用户体验，至少要避免灾难性故障。
 
**[TODO]** 开始前，请查看 [**98_VLM_Launch.ipynb**](./98_VLM_Launch.ipynb) 并执行启动单元。这将从 HuggingFace 加载一个**视觉语言模型（VLM）**，即[**微软的 Phi-3.5-vision-instruct 模型**](https://huggingface.co/microsoft/Phi-3.5-vision-instruct)，并将其部署到服务器。启动后，该服务应该可以通过入口从任何 notebook 访问。
- **为什么放在另一个 notebook 中？** 服务需要自己的事件循环以继续在后台运行。在 Jupyter notebook 的 Python 内核中管理多个事件循环可能过于繁琐。通过使用一个单独的 notebook，我们可以将其内核专门用于运行服务，确保顺利运行而不干扰主 notebook 的执行。

In [ ]:
## RUN THIS LINE TO LOCK IN SOME VARIABLES FOR LATER, INCLUDING SOME USED BY DEFAULT IN SOME CONNECTORS
model_path="http://localhost:9000/v1"
%env NVIDIA_BASE_URL=$model_path
%env NVIDIA_DEFAULT_MODE=open

In [ ]:
import requests

## Send a GET request to the LLM server port that lists the available models
model_discovery = requests.get(f"{model_path}/models").json()
model_name = model_discovery.get("data", [{}])[0].get("id")
model_discovery

### **调用 LLM**

现在我们已经部署了模型，为确认服务正在运行，可以尝试向它发送一些推理请求！这个服务的部署大致遵循 [**OpenAI 推理 API schema**](https://platform.openai.com/docs/guides/text-generation)，这是生态系统中相当标准的做法。我们的第一种方法应该是使用现成的连接器。

我们可以使用一个原始客户端，比如官方的 [**`OpenAI` python 客户端**](https://github.com/openai/openai-python)。这个客户端旨在以相对可移植的方式连接到服务部署，并可以作为基本调用单元从多种其它框架中调用。

In [ ]:
import os
from openai import OpenAI

client = OpenAI(
    base_url=model_path,
    api_key="None",
)

client.completions.create(
    prompt="Hello! How's it going",
    model=model_name,
)

或者，我们也可以使用一个位于基础客户端上的连接器，增加额外的假设和功能。它的目的是简化基础客户端的工作流，以用于特定用例，或与其它框架进行更自然的交互。本课程中，我们将在下一个 notebook 中使用 [**LangChain**](https://www.langchain.com)，并将受益于 [**NVIDIA 的 LangChain 连接器，ChatNVIDIA**](https://python.langchain.com/docs/integrations/chat/nvidia_ai_endpoints/)，我们先来熟悉一下： 

In [ ]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA

llm = ChatNVIDIA(
    model=model_name,
    max_tokens=4096,
)

print(repr(llm.invoke("Hello! How's it going")))

通过两个不同的入口调用服务器后，您可能注意到不同结果：**OpenAI 的补全请求给您提供了续写句子的功能，而 `ChatNVIDIA` 则给您了一个聊天式的响应。** 这是因为两个连接器调用了不同的服务入口，它们的功能也不同。`ChatNVIDIA` 在底层默认将输入发送到 `/chat/completions` 入口，并提供 `messages` 参数，而不是上面看到的 `prompt` 参数：

In [ ]:
llm._client.last_inputs

这在将输入传递给 LLM 之前应用了一个聊天模板。这个格式在训练过程中通过**指令微调（instruction fine-tuning）**和**合成数据生成（synthetic data generation）**得到了强化。模板可以被修改以创建任意风格并接受任意类型的消息，但一些典型的组件包括：
- **系统消息：**一个整体指令，不遵循聊天格式，概述模型的预期行为。以这种格式指定的指令通常在训练期间得到很大的强化，对模型表现有很大影响。
- **人类消息：**这些是聊天式或指令式的消息，旨在指导 LLM。它们通常会被系统消息补充（或可能被其压倒），是终端用户与 LLM 产品沟通的主要方式。
- **AI 消息：**这些通常紧随人类消息之后，可以是由 LLM 生成的，或者作为**少样本提示**的典型填充示例。

给定一系列消息，您的模型的实际输入如下所示：

In [ ]:
from jinja2 import Environment, FileSystemLoader

env = Environment(loader = FileSystemLoader('.'))
template = env.get_template('phi35.jinja')
output = template.render(
    messages = [
        {"role": "user", "content": "Hello! How's it going"}  ## Comment out to see longer-form
    ] or [
        {"role": "system", "content": "System Instruction"},
        {"role": "user", "content": "Hello! How's it going?"},
        {"role": "assistant", "content": "Very good! How about you?"},
        {"role": "user", "content": "Life is good!"},
    ], 
    add_generation_prompt=True
)
print(output)

LLM 以对话的方式进行响应，它正是这样被训练出来的。这也是为什么 "/chat/completions" 接口会返回类似下面的 "chat.completion" 响应：

In [ ]:
## OPTIONAL: Send the request directly through the requests POST interface
requests.post(**{
    **llm._client.last_inputs, 
    ## OPTIONAL: Changing the messages to for a multi-turn example
    # "json" : {**llm._client.last_inputs["json"], "messages": [
    #     {"role": "system", "content": "Please be a helpful assistant."},
    #     {"role": "user", "content": "Tell me about cats!"},
    #     {"role": "assistant", "content": "Cats are cool! Too cool for school!"},
    #     {"role": "user", "content": "How about dogs?"},
    # ]}
}).json()

# llm._client.last_response.json()

<div><img src="imgs/llm-router.png" width="800"/></div>

### **传入图像**

回想一下我们提到的多模态练习，其中提到了解码器专用的视觉语言模型，将图像投射到文本解码器的输入空间。我们提到这些模型通常较大，并且需要特别的训练才能正常工作。无论开发人员具体是如何实现的，他们都遵循了 [**OpenAI Vision API**](https://platform.openai.com/docs/guides/vision)，使模型能在更大的生态中良好运作。简而言之，我们可以用以下格式调用这个模型特定的能力：

In [ ]:
import requests
import base64

invoke_url = "http://localhost:9000/v1/chat/completions"
stream = False

with open("./img-files/paint-cat.jpg", "rb") as f:
  image_b64 = base64.b64encode(f.read()).decode()

headers = {
    "Authorization": "Bearer $API_KEY_REQUIRED_IF_EXECUTING_OUTSIDE_NGC",
    "Accept": "text/event-stream" if stream else "application/json"
}

payload = {
    "model": 'microsoft/phi-3.5-vision-instruct',
    "messages": [
        {'role': 'system', 'content': 'Please describe this picture.'},
        {'role': 'user', 'content': [
            {'type': 'image_url', 'image_url': {'url': f'data:image/jpeg;base64,{image_b64}', 'detail': 'low'}}
        ]},
    ],
    "max_tokens": 512,
    "temperature": 0.20,
    "top_p": 0.70,
    "stream": stream
}

response = requests.post(invoke_url, headers=headers, json=payload)

if stream:
    for line in response.iter_lines():
        if line:
            print(line.decode("utf-8"))
else:
    print(response.json())

<br>

这引出了几个有趣的问题： 

#### **提示模板似乎不支持图像！它们是如何传递的？** 

实际上，提示模板并不是“不”支持图像。从服务的角度来看，用户消息的内容是一个字典的列表。不过，tokenizer 和 embedder 会处理这些内容，将图像处理并投影到 LLM 的输入空间。这一切都隐藏在标准接口后面，因此这个推理服务能与 [**OpenAI 的 GPT-4o**](https://openai.com/index/hello-gpt-4o/)、[**NVIDIA 的 NVLM**](https://arxiv.org/abs/2409.11402) 以及开源的 [**Llama 3.2 (2024)**](https://ai.meta.com/blog/llama-3-2-connect-2024-vision-edge-mobile-devices/) 模型进行互操作，尽管它们的实现有所不同。

#### **如果没有提供图像，VLM 会遇到很多麻烦。这是*幻觉*吗？** 

幻觉指的是由于各种问题导致的不正确和不可预测的生成。它最常发生的原因包括以下几种：
- **模型的输入或生成超出了训练/微调的分布。**
    - 这可能包括过长的输入、过于复杂的指令、样本质量差的输出，或冲突的指令/格式。
- **模型没有足够的信息来做出合理的决定。**
    - 这包括指令不足或缺乏上下文以产生连贯的响应。
 
在这种情况下，询问一个图像而不提供图像会使模型的输入超出其训练/微调的领域，导致响应变得无意义。应该对模型进行额外的微调，以理解缺少图像输入的情况，但在客户端甚至服务端也可以采取额外措施，防止这种超出分布（OOD）事件的发生。

<hr>
<br>

## **6.3：** 快速并发处理

推理服务的一个好处是能通过轻量级接口轻松连接多种上下文。我们在通过入口连接的过程中看到了这一点，可以假设其它场景的过程也同样简单。从更微妙的角度来看，我们的连接在很大程度上是独立和非阻塞的，允许许多用户和应用同时连接到服务。

### 使用 vLLM 的并发性

为了说明这一点，请注意我们是怎么跟 notebook 4 中的解码器流示例相同的方式生成非常简单的流，该流会在生成后立即返回响应块（一个或多个 token）：

In [ ]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA

llm = ChatNVIDIA(
    model=model_name,
    max_tokens=4096,
)

for chunk in llm.stream("Tell me about birds! A few sentences please."):
    # print(repr(chunk))
    print(chunk.content, end="")

<br>

这是一个适用于实时应用的有用接口，单用户环境中的实现已经展示过了。不过，这种行为可以从多个实例同时调用，对性能的影响相对较小。

In [ ]:
from aiostream.stream import merge as stream_merge
from IPython.display import clear_output

streams = [
    llm.astream("Tell me about fish! One sentence please.", max_tokens=100),
    llm.astream("Tell me about birds! 1 paragraph please.", max_tokens=300),
    llm.astream("Tell me about dogs! 3 paragraphs please.", max_tokens=500),
    llm.astream("Tell me about cats! 5 paragraphs please."),
]
buffers = {}
async with stream_merge(*streams).stream() as streamer:
    async for chunk in streamer:
        buffers[chunk.id] = buffers.get(chunk.id, "") + chunk.content.replace("\n", " ")
        clear_output(wait=True)
        for buffer in buffers.values():
            print(buffer, end="\n\n")

<br>

不仅独立的缓冲区可以独立生成和返回结果，请求还可以由于一种称为“实时批处理（in-flight batching）”的机制而在不同时间到达，这使得预填充或自回归调用能在一组活动线程中分配，平滑处理优先级。

In [ ]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA
from tqdm.auto import tqdm
import asyncio

llm = ChatNVIDIA(
    model=model_name,
    max_tokens=128,
)

topics = [
    "birds", "cats", "dogs", "lizards", "hamsters", "dragons", 
    "fireworks", "GPUs", "happiness", "sadness", "42", "24", "infinity",
    "elephants", "snakes", "rabbits", "stars", "planets", "oceans",
    "mountains", "clouds", "rain", "sunshine", "snow", "ice",
    "trees", "flowers", "rivers", "lakes", "forests", "deserts",
    "music", "dance", "art", "technology", "science", "history",
    "poetry", "philosophy", "love", "fear", "adventure", "solitude",
    "friendship", "chaos", "order", "energy", "time", "space",
]

# Creating a list of tasks for asynchronous execution
tasks = [llm.ainvoke(f"Tell me about {topic}! 100 words!") for topic in topics]

# Processing tasks with a progress bar
async for task in tqdm(asyncio.as_completed(tasks), total=len(tasks)):
    response = await task
    print(response.content[:84].strip(), end="...\n")

<br>

### 进一步优化我们的部署

我们可以看到，通过简单的 vLLM 部署，我们已经能在相当数量的并发任务中实现合理的推理速度。对于个人开发者甚至是几个活跃用户来说，这种规模的模型足够应对多种任务。然而，我们的部署在用户、模型规模和任务复杂度的扩展方面仍有很大的改进空间。具体来说，有几个我们尚未触及的优化：

- **量化：**将模型权重的精度从浮点数变成整数值，可以显著减少内存使用并提高推理速度。这对于在边缘设备或计算资源有限的环境中进行部署特别有利，但需要一些时间进行量化，并可能对硬件有些要求。
- **推理设置：**微调部署参数，如部署/推理模式、层融合设置和资源分配，可以根据特定用例在灵活性和功能之间进行权衡。
- **框架优化：**分析当前系统并根据平台选择最快的框架，以最大化整体的效率。

这些优化的目的在于在给定预算内在速度、并发和灵活性之间进行权衡，并根据预期的实际工作负载调整设置。然而，由于我们当前的环境只是众多可能配置中的一种，可能并不代表您的用例或预算，因此我们将放弃手动 vLLM 量化和配置，而是用一个运行中的 NIM 微服务来补充环境。

具体来说，我们将使用 **Llama-3.1-8B 模型**作为默认的适中规模的 LLM 模型。该服务是在您的 Jupyter 环境启动时启动的，并且已针对系统进行了预先优化。

In [ ]:
## USE THIS ONE FOR GENERAL USE AS A SMALL-BUT-PURPOSE CHAT MODEL BEING RAN LOCALLY VIA NIM
from langchain_nvidia_ai_endpoints import ChatNVIDIA
model_path="http://nim:8000/v1"
model_name = requests.get(f"{model_path}/models").json().get("data", [{}])[0].get("id")
%env NVIDIA_BASE_URL=$model_path

llm = ChatNVIDIA(
    model=model_name,
    max_tokens=4096,
)

for chunk in llm.stream("Tell me about birds! A few sentences please."):
    print(chunk.content, end="")

尽管测试用例可能没有显示，但这个模型在**长上下文推理**、**复杂的聊天交互** 和**保留格式的指令跟随**方面应该会有显著的改善，具体内容将在后面探讨。

<br>

### 超出我们的环境

在整个课程中，您可能会发现设备上的 LLM 选项对于某些有趣的任务并不够用。因此，我们还将提供外部 API 服务的访问权限，以便可以尝试更大的模型配置。对于这项服务，您将能够连接到 [**`build.nvidia.com`**](https://build.nvidia.com/explore/discover) 中的某些模型，该网站本身托管在自动扩展集群上部署的 NVIDIA NIM 实例。您可能会对 `meta/llama-3.1-70b-instruct` 和 `meta/llama-3.1-405b-instruct` 模型感兴趣，它们的性能会比现在的 8B 配置有相当的提升。

<div><img src="imgs/api-options.png" width="800"/></div>

请注意，由于 [**`build.nvidia.com`**](https://build.nvidia.com/explore/discover) 入口是共享的，供多个用户使用，并且是为试用而设计的，因此某些模型可能会偶尔变慢。这种减速应该仅限于每个单独的模型部署，因此如果发生此事件，您可以随意尝试列表中的其它模型。

In [ ]:
# ## USE THIS ONE FOR ACCESS TO CATALOG OF RUNNING NIM MODELS IN `build.nvidia.com`
model_path="http://llm_client:9000/v1"

import requests

model_name = requests.get(f"{model_path}/models").json().get("data", [{}])[0].get("id")
%env NVIDIA_BASE_URL=$model_path
%env NVIDIA_DEFAULT_MODE=open

if "llm_client" in model_path:
    %env NVIDIA_MODEL_NAME=meta/llama-3.1-405b-instruct
else:
    %env NVIDIA_MODEL_NAME=$model_name

llm = ChatNVIDIA(
    model=model_name,
    max_tokens=4096,
)

for chunk in llm.stream("Tell me about birds! A few sentences please."):
    print(chunk.content, end="")

<br>

在接下来的 notebook 中，我们建议使用 `nim` 服务作为一个专用资源。如果您有想实验的内容，可以随意切换到 `llm_client`。下面的代码块允许您在各种选项之间切换:

In [ ]:
## USE THIS ONE TO START OUT WITH. NOTE IT'S INTENTED USE AS A VISUAL LANGUAGE MODEL FIRST
# model_path="http://localhost:9000/v1"
## USE THIS ONE FOR GENERAL USE AS A SMALL-BUT-PURPOSE CHAT MODEL BEING RAN LOCALLY VIA NIM
model_path="http://nim:8000/v1"
# ## USE THIS ONE FOR ACCESS TO CATALOG OF RUNNING NIM MODELS IN `build.nvidia.com`
# model_path="http://llm_client:9000/v1"

model_name = requests.get(f"{model_path}/models").json().get("data", [{}])[0].get("id")
%env NVIDIA_BASE_URL=$model_path
%env NVIDIA_DEFAULT_MODE=open

if "llm_client" in model_path:
    model_name = "meta/llama-3.1-70b-instruct"

llm = ChatNVIDIA(
    model=model_name,
    max_tokens=4096,
)

for chunk in llm.stream("Tell me about birds! A few sentences please."):
    print(chunk.content, end="")

In [ ]:
# llm._client.last_inputs

<hr>
<br>

## **6.4：** 深入文本生成

正如您所见，我们的 LLM 服务器是一个多功能工具，能够处理各种文本生成任务，这得益于其灵活的接口。现在让我们利用这些能力来探索一些现实世界中潜在的任务，看看我们的模型表现如何。在这个练习中，我们将采用递归的方法，将本 notebook 作为上下文进行加载：

In [ ]:
# !wget https://huggingface.co/nvidia/Llama-3.1-Minitron-4B-Width-Base/resolve/main/tokenizer.json
from transformers import PreTrainedTokenizerFast
from chatbot.jupyter_tools import FileLister
import os

llama_tokenizer = PreTrainedTokenizerFast(tokenizer_file="tokenizer.json", clean_up_tokenization_spaces=True)
filenames = [v for v in sorted(os.listdir("temp_dir")) if v.endswith(".ipynb")]
context_files = ["06_llm_server.ipynb"]
full_context = FileLister().to_string(files=context_files, workdir=".")

print("Full context character length:", len(full_context))
print("Full context token length:", len(llama_tokenizer.encode(text=full_context)))

<br>

虽然我们的 VLM 可能在处理如此长的输入时会遇到问题，但 Llama 模型应该足够应对其中*一些*有趣的任务——前提是我们能正确使用它。为此，我们将使用一个叫做 [**LangChain**](https://python.langchain.com/docs/tutorials/) 的 LLM 编排框架，以简化模型的使用，并直接投入到开发有趣的 LLM 工作流中。

**现在，让我们来制定 LLM 的指令。** 为了方便这个过程，我们将使用 LangChain 这个 LLM 编排框架来简化我们的状态管理工作流。具体来说：
- 我们将使用 `ChatPromptTemplate` 来结构化我们的消息，以引导 LLM 理解上下文并生成相应的回复。这个结构期待一个输入变量的字典（可能包括一系列消息，如 `placeholder` 中所示），并将其格式化为 LLM 输入。
- 我们将像之前一样使用 `ChatNVIDIA` 连接器，知道它接受字符串或消息列表，并返回 `AIMessage`。
- 最后，我们将使用 `StrOutputParser` 组件，自动提取生成的 `AIMessage` 中的内容。

在 LangChain 中，这些组件都被视为**“运行时（runnable）”**，或者说是可以与其它组件链接的特殊 lambda 函数。它们又可以组合成另一个运行时，通过 `invoke` 和 `stream` 这样的方式将输入从一个传递到下一个。

**注意：** 从表面上看，运行时可能只是一种花哨的 lambda。然而，实际上它们还能实现异步路由和生成器堆叠，以便让它们的逻辑能够通过流和异步过程传递。

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

llm = ChatNVIDIA(
    model=model_name,
    max_tokens=8000,
)

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", (
        "You are a helpful DLI Chatbot who can request and reason about notebooks."
        " Be as concise as necessary, but follow directions as best as you can."
        " Please help the user out by answering any of their questions and following their instructions."
    )),
    ("human", "Here is the notebook I want you to work with: {full_context}. Remembering this, start the conversation over."),
    ("ai", "Awesome! I will work with this as context and will restart the conversation."),
    ("placeholder", "{messages}")
])

pipeline = chat_prompt | llm | StrOutputParser()

## GIVEN: See if it can do something short with full understanding of context [Long-Context Reasoning]
## TASKS: Try to see what happens when you try long-form generation, code generation, etc.

state = {
    "filenames": filenames,
    "context_files": context_files, 
    "full_context": full_context,
    "messages": [("human", "Can you give me a summary of the notebook?")]
}

for chunk in pipeline.stream(state):
    print(chunk, end="")

<br>

有了这些工具，就可以来挑战我们的 LLM 生成 notebook 摘要，进行关于其内容的对话，甚至生成扩展我们探索主题和思想的创意输出。

**[练习] 在上面的代码单元中，尝试让您的 LLM 工作流进入以下情况：**
- **短输入 短输出**
- **长输入 短输出**
- **短输入 长输出**
- **长输入 长输出**
- **代码输出（Python, SQL 等）**

这些上下文的影响将在下一个 notebook 的开头进行更详细的讨论。

<div><img src="imgs/basic-chat.png" width="800"/></div>

### [练习] 创建对话循环

我们现在知道如何通过状态输入将命令发送到 LLM 链，那么为什么不更进一步，创建一个简单的用户界面呢？在接下来的单元中，添加必要的流式处理和状态管理逻辑，以启用与您之前定义的 LLM 链的多轮对话。
- 可以像之前一样假设静态上下文。
- 可以使用 `("human"/"ai", message_body)` 语法来定义消息。
- 确保利用运行状态（running state）。

In [ ]:
filenames = [v for v in sorted(os.listdir("temp_dir")) if v.endswith(".ipynb")]
context_files = ["06_llm_server.ipynb"]
full_context = FileLister().to_string(files=context_files, workdir=".")

## Initialize the state to have the full context and an initial list of messages
state = {
    "filenames": filenames,
    "context_files": context_files, 
    "full_context": full_context,
    "messages": []
}

while True:
    try: 
        ## Initiate an agent buffer to accumulate agent response
        agent_msg = ""

        human_msg = input("\n[Human]: ")
        ## TODO: Update the messages appropriately
        
        print("\n[Agent]: ", end="")
        ## TODO: Stream the LLM's response directly to output and accumulate it

        ## TODO: Update the messages list appropriately
        
    except KeyboardInterrupt:
        print("KeyboardInterrupt")
        break

<details>
<summary><b>参考答案</b></summary>

```python
## TODO: Update the messages appropriately
state["messages"] += [("human", human_msg)]<br>
## TODO: Stream the LLM's response directly to output and accumulate it
for token in pipeline.stream(state):
    agent_msg += token
    print(token, end="")<br>
## TODO: Update the messages list appropriately
state["messages"] += [("ai", agent_msg)]
```

</details>

<hr>
<br>

# <font color="#76b900">**总结**</font>

在这个 notebook 中，我们探讨了如何通过利用 LLM 服务将我们的模型扩展到实际应用场景。我们首先理解了之前模型在性能、效率和可扩展性方面的局限性。为了应对这些挑战，引入了 LLM 服务的概念，并演示了它们如何用于部署更强大的模型，这些模型支持任意指令的执行和对话。

### 关键点：

- GenAI 服务启动例程可以成为更具扩展性部署的起点。
- vLLM 和 NIM 是很好的起点，并且可以简单地与标准化 API 和预优化选项配合使用。
- 扩展部署允许并发请求和用户查询特定模型以满足各种用例。
- 像 LangChain 这样的客户端软件堆栈，使得以入口为中心的这种易用工作流得以实现，从而允许客户端编排。

### 下一步：

**在下一个 notebook 中，我们将建立在这些基础之上，深入研究 LLM 的编排。**我们将探索提示工程、检索、路由、工具和智能体的基本原理，帮助您掌握创建强大且高效的 LLM 应用所需的知识和工具。

In [ ]:
# ## Please Run When You're Done!
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)